The strategies like Bollinger Band CrossOver, RSI Based Momentum Trading and MA Crossover have key shortcommings, especially in a Bull Market that India is currently in: 

- **RSI** is better at identifying reversals rather than continuation patterns. In bull markets, RSI might suggest corrections that never materialize, early exits
- **RSI** is unreliable in volatile markets
- **Bollinger Bands** work well only in sideways markets and cause early exits in breakout
-  **Bollinger Bands** are better at capturing **short-term volatility** and **mean-reverting conditions**. In strong trending markets (like a bull market), the bands widen, but this doesn’t necessarily indicate when the trend will end
- In volatile markets, the price action happens in a very short time and since RSI and MA are lagging indicators, they miss majority of the action

## Fetching Past 20Year NIFTY-50 Data

In [17]:
import yfinance as yf
import pandas as pd
import numpy as np
import talib as ta
from ta.volatility import BollingerBands
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

In [6]:
df = pd.DataFrame(yf.download("^NSEI", start="2004-07-01", end="2024-07-01"))

[*********************100%***********************]  1 of 1 completed


In [4]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0
2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0
2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0
2007-09-20,4734.850098,4760.850098,4721.149902,4747.549805,4747.549805,0
2007-09-21,4752.950195,4855.700195,4733.700195,4837.549805,4837.549805,0


In [5]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-06-24,23382.300781,23558.099609,23350.000000,23537.849609,23537.849609,239400
2024-06-25,23577.099609,23754.150391,23562.050781,23721.300781,23721.300781,298100
2024-06-26,23723.099609,23889.900391,23670.449219,23868.800781,23868.800781,287800
2024-06-27,23881.550781,24087.449219,23805.400391,24044.500000,24044.500000,515200
2024-06-28,24085.900391,24174.000000,23985.800781,24010.599609,24010.599609,354800


In [7]:
df.isnull().sum().any()

False

## The Strategy

## Implementing Strategy 

In [12]:
def EMA(data, window):
    return ta.ema(data['Close'], window)

In [11]:
def RSI(data, window, _=None, __=None, ___=None):
    return ta.rsi(data['Close'], window)

In [14]:
def BBAND_L(data, _,  column_name, window,  std):
    return BollingerBands(close=data[column_name], window=window, window_dev=std).bollinger_lband()

In [15]:
def BBAND_H(data, _, column_name, window,  std):
	return BollingerBands(close=data[column_name], window=window, window_dev=std).bollinger_hband()

In [16]:
def calculate_trend(values):
    if len(values==0):
        return 0, 0
    x = np.arange(1, len(values)+1, 1)
    y = np.array(values)
    
    x_new = x[~np.isnan(y)]
    y_new = y[~np.isnan(y)]
    
    m,c = np.polyfit(x_new, y_new, 1)
    
    return m, c

In [ ]:
class MyStrategy(Strategy):
    
    long_ema_period = 365
    rsi_period = 14
    rsi_lookback = 5
    bbands_period = 20

    bbands_std1 = 2.6
    bbands_std2 = 1.6
	bbands_std3 = 2.0
 
	last_trade_index = 0
 	last_trade_price = 0
	
	take_profit = 0
	stop_loss = 10
  
	long_hold = 0
 
	i = 0
    
    def init(self):
        super().init()
        
        # Indicators
        self.ema1 = self.I(EMA, self.data.df, self.long_ema_period)
        self.bollinger_close_lband = self.I(BBAND_L, self.data.df, None, 'Close', self.bbands_period, self.bbands_std1)
        self.bollinger_close_hband = self.I(BBAND_H, self.data.df,None, 'Close', self.bbands_period, self.bbands_std2)
        
        # Signals
        self.data.df['RSI'] = RSI(df, self.rsi_period, None, None, None)
        self.rsi_bollinger_bands = self.I(lambda *args: (RSI(*args), BBAND_L(*args), BBAND_H(*args)), self.data.df, self.rsi_period, "RSI", self.rsi_lookback, self.bbands_period, self.bbands_std3)
        
    def next(self):
        
        self.i += 1
        
        if (self.i < self.long_ema_period):
			return

		long_entry_signals = 0
		long_exit_signals = 0
  
		# Long Term Trend Filter
		ema1_lb = self.ema1[-20:]
		ema1_m, _ = calculate_trend(ema1_lb)
  
		if ema1_m >= 0.01:
			long_entry_signals += 1
   
		# Long Entry
		if self.data.Close[-1] >= self.bollinger_close_lband[-1] and self.data.Close[-2] < self.bollinger_close_lband[-2]:
			long_entry_signals += 1
   
		rsi = self.rsi_bollinger_bands
		rsi_lb = rsi[(-1*self.rsi_lookback):]
		bollinger_rsi_lband = self.rsi_bollinger_bands[1]
		bollinger_rsi_lb = bollinger_rsi_lg
        
        
        
